In [ ]:
import numpy as np
import pickle
from collections import defaultdict, Counter, deque
import random
from typing import List, Set, Tuple, Dict
import re

# ADD THIS FUNCTION
def _create_nested_float_dict():
    return defaultdict(float)

In [ ]:
import numpy as np
import pickle
from collections import defaultdict, Counter, deque
import random
from typing import List, Set, Tuple, Dict
import re

class AdvancedHMMHangman:
    """Enhanced HMM with word filtering for superior prediction"""

    def __init__(self):
        self.word_length_models = {}
        self.words_by_length = defaultdict(list)
        self.letter_frequencies = None
        self.alphabet = set('abcdefghijklmnopqrstuvwxyz')
        self.bigram_freq = defaultdict(lambda: defaultdict(int))
        self.trigram_freq = defaultdict(lambda: defaultdict(int))

    def train(self, corpus_file: str):
        """Train enhanced HMM with multiple pattern sources"""
        print("Training Advanced HMM with word filtering...")

        with open(corpus_file, 'r') as f:
            words = [line.strip().lower() for line in f if line.strip()]

        # Group words by length
        for word in words:
            if word.isalpha():
                self.words_by_length[len(word)].append(word)

        # Global letter frequencies
        all_letters = ''.join(words)
        letter_counts = Counter(all_letters)
        total = sum(letter_counts.values())
        self.letter_frequencies = {l: letter_counts[l] / total for l in self.alphabet}

        # Build n-gram models
        for word in words:
            if not word.isalpha():
                continue
            # Bigrams
            for i in range(len(word) - 1):
                self.bigram_freq[word[i]][word[i+1]] += 1
            # Trigrams
            for i in range(len(word) - 2):
                context = word[i:i+2]
                self.trigram_freq[context][word[i+2]] += 1

        # Normalize bigrams
        for c1 in self.bigram_freq:
            total = sum(self.bigram_freq[c1].values())
            for c2 in self.bigram_freq[c1]:
                self.bigram_freq[c1][c2] /= total

        # Normalize trigrams
        for context in self.trigram_freq:
            total = sum(self.trigram_freq[context].values())
            for letter in self.trigram_freq[context]:
                self.trigram_freq[context][letter] /= total

        # Train positional models
        for length, word_list in self.words_by_length.items():
            self.word_length_models[length] = self._train_positional_model(word_list, length)

        print(f"Trained models for {len(self.word_length_models)} word lengths")
        print(f"Total vocabulary: {sum(len(v) for v in self.words_by_length.values())} words")

    def _train_positional_model(self, words: List[str], length: int) -> Dict:
        """Train position-based probability model"""
        model = {'position_probs': defaultdict(_create_nested_float_dict)}

        for word in words:
            for pos, letter in enumerate(word):
                model['position_probs'][pos][letter] += 1

        # Normalize with smoothing
        for pos in model['position_probs']:
            total = sum(model['position_probs'][pos].values()) + 26  # Add-one smoothing
            for letter in self.alphabet:
                count = model['position_probs'][pos].get(letter, 0) + 1
                model['position_probs'][pos][letter] = count / total

        return model

    def _build_regex_pattern(self, masked_word: str, guessed_letters: Set[str]) -> re.Pattern:
        """Build regex pattern for word filtering"""
        wrong_letters = guessed_letters - set(masked_word)
        pattern_parts = []

        for char in masked_word:
            if char == '_':
                if wrong_letters:
                    negation = ''.join(sorted(wrong_letters))
                    pattern_parts.append(f"[^{negation}]")
                else:
                    pattern_parts.append('.')
            else:
                pattern_parts.append(char)

        return re.compile('^' + ''.join(pattern_parts) + '$')

    def predict_probabilities(self, masked_word: str, guessed_letters: Set[str]) -> Dict[str, float]:
        """Predict using hybrid word-filtering + statistical approach"""
        length = len(masked_word)
        remaining_letters = self.alphabet - guessed_letters

        if not remaining_letters:
            return {l: 0.0 for l in self.alphabet}

        # STRATEGY 1: Word List Filtering (Most Powerful)
        if length in self.words_by_length:
            pattern = self._build_regex_pattern(masked_word, guessed_letters)
            matching_words = [w for w in self.words_by_length[length] if pattern.match(w)]

            if matching_words:
                letter_counts = Counter()
                for word in matching_words:
                    for i, char in enumerate(word):
                        if masked_word[i] == '_':
                            letter_counts[char] += 1

                total = sum(letter_counts.values())
                if total > 0:
                    probs = {}
                    for letter in remaining_letters:
                        probs[letter] = letter_counts.get(letter, 0) / total
                    return probs

        # STRATEGY 2: Combined Statistical Approach
        probs = defaultdict(float)

        for letter in remaining_letters:
            score = 0.0
            weight = 0.0

            # Positional probabilities
            if length in self.word_length_models:
                model = self.word_length_models[length]
                for pos, char in enumerate(masked_word):
                    if char == '_':
                        pos_prob = model['position_probs'][pos].get(letter, 0.001)
                        score += pos_prob * 3.0
                        weight += 3.0

            # Context-based (bigram/trigram)
            for pos, char in enumerate(masked_word):
                if char == '_':
                    # Left context (bigram)
                    if pos > 0 and masked_word[pos-1] != '_':
                        left = masked_word[pos-1]
                        bigram_prob = self.bigram_freq[left].get(letter, 0.001)
                        score += bigram_prob * 2.5
                        weight += 2.5

                    # Right context (bigram)
                    if pos < length - 1 and masked_word[pos+1] != '_':
                        # Reverse lookup
                        right = masked_word[pos+1]
                        bigram_prob = self.bigram_freq[letter].get(right, 0.001)
                        score += bigram_prob * 2.5
                        weight += 2.5

                    # Trigram context
                    if pos > 0 and pos < length - 1:
                        left = masked_word[pos-1] if masked_word[pos-1] != '_' else None
                        right = masked_word[pos+1] if masked_word[pos+1] != '_' else None
                        if left and right:
                            context = left + right
                            trigram_prob = self.trigram_freq.get(context, {}).get(letter, 0.001)
                            score += trigram_prob * 2.0
                            weight += 2.0

            # Global frequency (baseline)
            score += self.letter_frequencies.get(letter, 0.001)
            weight += 1.0

            probs[letter] = score / weight if weight > 0 else 0.001

        # Normalize
        total = sum(probs.values())
        return {l: probs[l] / total for l in probs} if total > 0 else {l: 1.0/len(remaining_letters) for l in remaining_letters}


class HangmanEnvironment:
    """Optimized Hangman environment"""

    def __init__(self, word_list: List[str], max_wrong: int = 6):
        self.word_list = word_list
        self.max_wrong = max_wrong
        self.reset()

    def reset(self, word: str = None):
        self.target_word = word if word else random.choice(self.word_list)
        self.masked_word = '_' * len(self.target_word)
        self.guessed_letters = set()
        self.wrong_guesses = 0
        self.game_over = False
        self.won = False
        return self.get_state()

    def get_state(self) -> Dict:
        return {
            'masked_word': self.masked_word,
            'guessed_letters': self.guessed_letters.copy(),
            'wrong_guesses': self.wrong_guesses,
            'lives_left': self.max_wrong - self.wrong_guesses,
            'progress': 1.0 - (self.masked_word.count('_') / len(self.target_word))
        }

    def step(self, letter: str) -> Tuple[Dict, float, bool]:
        letter = letter.lower()

        if letter in self.guessed_letters:
            return self.get_state(), -20, self.game_over

        self.guessed_letters.add(letter)

        if letter in self.target_word:
            new_masked = list(self.masked_word)
            count = 0
            for i, char in enumerate(self.target_word):
                if char == letter:
                    new_masked[i] = letter
                    count += 1
            self.masked_word = ''.join(new_masked)

            if '_' not in self.masked_word:
                self.game_over = True
                self.won = True
                reward = 200 - (self.wrong_guesses * 10)
            else:
                reward = 15 * count  # Reward proportional to letters revealed
        else:
            self.wrong_guesses += 1
            reward = -25

            if self.wrong_guesses >= self.max_wrong:
                self.game_over = True
                self.won = False
                reward = -100

        return self.get_state(), reward, self.game_over


class ImprovedRLAgent:
    """Enhanced RL agent with better exploration and HMM integration"""

    def __init__(self, hmm: AdvancedHMMHangman, alpha: float = 0.15, gamma: float = 0.95):
        self.hmm = hmm
        self.q_table = defaultdict(lambda: defaultdict(float))
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = 1.0
        self.epsilon_decay = 0.9995
        self.epsilon_min = 0.01
        self.alphabet = set('abcdefghijklmnopqrstuvwxyz')

        # Experience replay for better learning
        self.memory = deque(maxlen=5000)
        self.visit_counts = defaultdict(lambda: defaultdict(int))

    def _state_key(self, state: Dict) -> str:
        return f"{state['masked_word']}:{state['lives_left']}"

    def choose_action(self, state: Dict, training: bool = True) -> str:
        available_letters = self.alphabet - state['guessed_letters']

        if not available_letters:
            return random.choice(list(self.alphabet))

        # Get HMM probabilities
        hmm_probs = self.hmm.predict_probabilities(state['masked_word'], state['guessed_letters'])

        # Exploration with smart bias toward HMM
        if training and random.random() < self.epsilon:
            # Weighted random selection (favor HMM suggestions even during exploration)
            if random.random() < 0.5:
                weights = [hmm_probs.get(l, 0.001) for l in available_letters]
                total = sum(weights)
                weights = [w/total for w in weights]
                return np.random.choice(list(available_letters), p=weights)
            else:
                return random.choice(list(available_letters))

        # Exploitation: Combine Q-values with HMM
        state_key = self._state_key(state)
        best_letter = None
        best_score = float('-inf')

        for letter in available_letters:
            q_value = self.q_table[state_key].get(letter, 0)
            hmm_prob = hmm_probs.get(letter, 0.001)

            # UCB-style exploration bonus
            visit_count = self.visit_counts[state_key][letter]
            exploration_bonus = np.sqrt(2 * np.log(sum(self.visit_counts[state_key].values()) + 1) / (visit_count + 1))

            # Combine signals
            combined_score = q_value + (hmm_prob * 100) + (exploration_bonus * 5 if training else 0)

            if combined_score > best_score:
                best_score = combined_score
                best_letter = letter

        return best_letter if best_letter else random.choice(list(available_letters))

    def update(self, state: Dict, action: str, reward: float, next_state: Dict, done: bool):
        state_key = self._state_key(state)
        next_state_key = self._state_key(next_state)

        # Track visits
        self.visit_counts[state_key][action] += 1

        # Q-learning update
        current_q = self.q_table[state_key][action]

        if done:
            max_next_q = 0
        else:
            available_letters = self.alphabet - next_state['guessed_letters']
            max_next_q = max([self.q_table[next_state_key].get(l, 0) for l in available_letters]) if available_letters else 0

        new_q = current_q + self.alpha * (reward + self.gamma * max_next_q - current_q)
        self.q_table[state_key][action] = new_q

        # Store experience
        self.memory.append((state, action, reward, next_state, done))

    def replay_batch(self, batch_size: int = 32):
        """Learn from random experiences"""
        if len(self.memory) < batch_size:
            return

        batch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in batch:
            self.update(state, action, reward, next_state, done)

    def train(self, env: HangmanEnvironment, episodes: int = 15000):
        print(f"\nTraining RL agent for {episodes} episodes...")

        wins = 0
        total_wrong = 0
        recent_wins = deque(maxlen=100)

        for episode in range(episodes):
            state = env.reset()
            done = False

            while not done:
                action = self.choose_action(state, training=True)
                next_state, reward, done = env.step(action)
                self.update(state, action, reward, next_state, done)
                state = next_state

            if env.won:
                wins += 1
                recent_wins.append(1)
            else:
                recent_wins.append(0)
            total_wrong += env.wrong_guesses

            # Batch replay every 10 episodes
            if episode % 10 == 0:
                self.replay_batch()

            # Decay epsilon
            self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

            if (episode + 1) % 1000 == 0:
                win_rate = wins / (episode + 1)
                recent_wr = sum(recent_wins) / len(recent_wins) if recent_wins else 0
                avg_wrong = total_wrong / (episode + 1)
                print(f"Episode {episode + 1}/{episodes} - Win Rate: {win_rate:.3f} | "
                      f"Recent 100: {recent_wr:.3f} | Avg Wrong: {avg_wrong:.2f} | Epsilon: {self.epsilon:.3f}")

        final_win_rate = wins / episodes
        print(f"\nTraining complete! Final win rate: {final_win_rate:.3f}")


def evaluate_agent(agent: ImprovedRLAgent, test_words: List[str], max_wrong: int = 6, verbose: bool = True) -> Dict:
    """Evaluate agent on test set"""
    if verbose:
        print(f"\nEvaluating on {len(test_words)} test games...")

    wins = 0
    total_wrong = 0
    total_repeated = 0

    for i, word in enumerate(test_words):
        env = HangmanEnvironment([word], max_wrong)
        state = env.reset(word)
        guessed_this_game = set()

        while not env.game_over:
            action = agent.choose_action(state, training=False)

            if action in guessed_this_game:
                total_repeated += 1
            guessed_this_game.add(action)

            state, reward, done = env.step(action)

        if env.won:
            wins += 1
        total_wrong += env.wrong_guesses

        if verbose and (i + 1) % 500 == 0:
            print(f"Evaluated {i + 1}/{len(test_words)} games...")

    success_rate = wins / len(test_words)
    final_score = (success_rate * len(test_words)) - (total_wrong * 5) - (total_repeated * 2)

    results = {
        'games_played': len(test_words),
        'wins': wins,
        'success_rate': success_rate,
        'total_wrong': total_wrong,
        'avg_wrong': total_wrong / len(test_words),
        'total_repeated': total_repeated,
        'final_score': final_score
    }

    if verbose:
        print("\n" + "="*60)
        print("EVALUATION RESULTS")
        print("="*60)
        print(f"Games Played: {results['games_played']}")
        print(f"Wins: {results['wins']}")
        print(f"Success Rate: {results['success_rate']:.2%}")
        print(f"Total Wrong Guesses: {results['total_wrong']}")
        print(f"Average Wrong Guesses: {results['avg_wrong']:.2f}")
        print(f"Total Repeated Guesses: {results['total_repeated']}")
        print(f"\nFINAL SCORE: {results['final_score']:.2f}")
        print("="*60)

    return results


if __name__ == "__main__":
    # Configuration
    CORPUS_FILE = "/content/corpus.txt"
    TEST_FILE = "/content/test.txt"
    TRAINING_EPISODES = 15000

    # Step 1: Train HMM
    print("Step 1: Training Advanced HMM")
    hmm = AdvancedHMMHangman()
    hmm.train(CORPUS_FILE)

    # Step 2: Load data
    print("\nStep 2: Loading training and test data")
    with open(CORPUS_FILE, 'r') as f:
        train_words = [line.strip().lower() for line in f if line.strip() and line.strip().isalpha()]

    with open(TEST_FILE, 'r') as f:
        test_words = [line.strip().lower() for line in f if line.strip() and line.strip().isalpha()]

    print(f"Training set: {len(train_words)} words")
    print(f"Test set: {len(test_words)} words")

    # Step 3: Train RL agent
    print("\nStep 3: Training Enhanced RL Agent")
    env = HangmanEnvironment(train_words)
    agent = ImprovedRLAgent(hmm)
    agent.train(env, episodes=TRAINING_EPISODES)

    # Step 4: Evaluate
    print("\nStep 4: Final Evaluation")
    results = evaluate_agent(agent, test_words)

    # Step 5: Save model
    print("\nSaving model...")
    with open('hangman_agent_optimized.pkl', 'wb') as f:
        pickle.dump({'hmm': hmm, 'agent': agent}, f)
    print("Model saved to 'hangman_agent_optimized.pkl'")

    # Additional analysis
    if results['success_rate'] >= 0.80:
        print("\n🎉 SUCCESS! Achieved 80%+ win rate!")
    else:
        print(f"\n⚠️  Current win rate: {results['success_rate']:.1%}. Target: 80%+")
        print("Consider: Longer training, hyperparameter tuning, or more sophisticated features")

Step 1: Training Advanced HMM
Training Advanced HMM with word filtering...
Trained models for 24 word lengths
Total vocabulary: 49979 words

Step 2: Loading training and test data
Training set: 49979 words
Test set: 2000 words

Step 3: Training Enhanced RL Agent

Training RL agent for 15000 episodes...
Episode 1000/15000 - Win Rate: 0.497 | Recent 100: 0.630 | Avg Wrong: 4.73 | Epsilon: 0.606
Episode 2000/15000 - Win Rate: 0.633 | Recent 100: 0.850 | Avg Wrong: 4.15 | Epsilon: 0.368
Episode 3000/15000 - Win Rate: 0.715 | Recent 100: 0.860 | Avg Wrong: 3.73 | Epsilon: 0.223
Episode 4000/15000 - Win Rate: 0.772 | Recent 100: 0.940 | Avg Wrong: 3.38 | Epsilon: 0.135
Episode 5000/15000 - Win Rate: 0.807 | Recent 100: 0.940 | Avg Wrong: 3.10 | Epsilon: 0.082
Episode 6000/15000 - Win Rate: 0.832 | Recent 100: 0.970 | Avg Wrong: 2.90 | Epsilon: 0.050
Episode 7000/15000 - Win Rate: 0.848 | Recent 100: 0.910 | Avg Wrong: 2.76 | Epsilon: 0.030
Episode 8000/15000 - Win Rate: 0.860 | Recent 100: 0

AttributeError: Can't get local object 'AdvancedHMMHangman._train_positional_model.<locals>.<lambda>'

In [4]:
import time
import sys

def play_game_demo(agent_to_demo, word_to_play):
    """
    Plays a single game of Hangman using the trained agent and prints the steps.

    Assumes 'HangmanEnvironment' class is already defined in the notebook.
    """
    print("="*40)
    print(f"DEMO: Agent will try to guess the word!")
    print(f"The word has {len(word_to_play)} letters.")
    print("="*40)

    # Use the HangmanEnvironment class already defined in your notebook
    game_env = HangmanEnvironment([word_to_play], max_wrong=6)
    state = game_env.reset(word=word_to_play)

    # Give a 2-second pause to start
    time.sleep(2)

    while not game_env.game_over:
        # 1. Get current state and print it
        masked = ' '.join(list(state['masked_word'])) # Add spaces for readability
        lives = state['lives_left']
        guessed = sorted(list(state['guessed_letters']))

        print(f"Word:  {masked}")
        print(f"Lives: {lives} ❤️")
        print(f"Guessed: {' '.join(guessed)}")

        # 2. Agent chooses action
        # We pass training=False so it doesn't explore and only uses its policy
        action = agent_to_demo.choose_action(state, training=False)

        print(f"\nAgent guesses: '{action}'")
        sys.stdout.flush() # Make sure text prints before sleep

        # 3. Take step in the environment
        state, reward, done = game_env.step(action)

        # 4. Print result of the guess
        if reward > 0 and not done:
            print("...Correct!")
        elif reward < -20: # This was a wrong guess
            print("...Incorrect!")
        elif reward == -20: # This was a repeated guess
             print("...Already guessed that!")

        time.sleep(1.5) # Pause for 1.5 seconds to make it readable
        print("-" * 20)

    # Game is over, print final result
    print("="*40)
    print("GAME OVER")
    if game_env.won:
        print(f"🎉 Agent WON! The word was '{word_to_play}'")
    else:
        print(f"😢 Agent LOST. The word was '{word_to_play}'")
    print(f"Final state: {game_env.masked_word}")
    print(f"Total wrong guesses: {game_env.wrong_guesses}")
    print("="*40)

In [6]:
import random

# The 'test_words' and 'agent' variables should still be in memory
# from the previous cells you ran.

if 'test_words' in locals() and 'agent' in locals():
    # Pick a random word from the test set
    demo_word = random.choice(test_words)

    # Run the demo
    play_game_demo(agent, demo_word)

    # You can also test a specific word:
    # play_game_demo(agent, "python")

else:
    print("Error: 'test_words' or 'agent' not found in memory.")
    print("Please make sure you have run all the previous cells in the notebook.")

DEMO: Agent will try to guess the word!
The word has 11 letters.
Word:  _ _ _ _ _ _ _ _ _ _ _
Lives: 6 ❤️
Guessed: 

Agent guesses: 'e'
...Correct!
--------------------
Word:  _ _ _ _ _ _ _ e _ _ _
Lives: 6 ❤️
Guessed: e

Agent guesses: 'n'
...Correct!
--------------------
Word:  _ _ _ _ _ _ _ e n _ _
Lives: 6 ❤️
Guessed: e n

Agent guesses: 'c'
...Correct!
--------------------
Word:  _ _ _ _ _ _ _ e n _ c
Lives: 6 ❤️
Guessed: c e n

Agent guesses: 'i'
...Correct!
--------------------
Word:  _ _ _ _ _ _ _ e n i c
Lives: 6 ❤️
Guessed: c e i n

Agent guesses: 'o'
...Correct!
--------------------
Word:  _ _ _ _ _ o _ e n i c
Lives: 6 ❤️
Guessed: c e i n o

Agent guesses: 'g'
...Correct!
--------------------
Word:  _ _ _ _ _ o g e n i c
Lives: 6 ❤️
Guessed: c e g i n o

Agent guesses: 't'
...Correct!
--------------------
Word:  t _ _ _ _ o g e n i c
Lives: 6 ❤️
Guessed: c e g i n o t

Agent guesses: 'r'
...Incorrect!
--------------------
Word:  t _ _ _ _ o g e n i c
Lives: 5 ❤️
Guessed: c 